In [20]:
import openai
import os
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain
import pandas as pd
from langchain.llms import OpenAI
os.environ["OPENAI_API_KEY"] = input("请输入API-KEY")
os.environ["http_proxy"] = "http://localhost:7890"
os.environ["https_proxy"] = "http://localhost:7890"
# 用于加入搜索功能
# os.environ["SERPAPI_API_KEY"] = "75fedecc5cd9801954afda5610ac7801e3ab7eeb01d9ae2fe393a00616e7d40a"

In [21]:
# # 这是一段试试api连接的代码，可以用于测试llm是否给出回应
# llm = OpenAI(temperature=0.9)
# text = "What would be a good company name for a company that makes colorful socks?"
# print(llm(text))

In [22]:
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
# 建立prompt模板
template="You are my personal secretary assisting me in planning my schedule. Here is my current schedule: {schedule}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="Now, here are my requirements: {intent}. Please fulfill the above requirements, make modifications to my schedule as necessary while maintaining the original format, and provide the updated schedule at the beginning. Your response should be strictly divided into two sections:""Schedule:"" and ""Answer:""."
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])


In [23]:
def extract_answer_and_schedule(text):
    # 提取回答内容
    schedule_start = text.index("Schedule:") + len("Schedule:")
    schedule_end = text.index("Answer:")
    schedule = text[schedule_start:schedule_end].strip()

    # 提取日程表内容
    answer_start = text.index("Answer") + len("Answer:")
    answer = text[answer_start:].strip()

    return answer, schedule

In [24]:
# 定义一个函数，用于将AI秘书回复的‘日程表’字符串转换为dataframe格式，并输出至csv
import pandas as pd

def parse_string_to_dataframe(string,filename):
    lines = string.strip().split('\n')  # 将字符串拆分成行
    headers = lines[0].split()  # 获取表头
    data = [line.split(maxsplit=3) for line in lines[1:]]  # 拆分行并限制拆分为4个部分
    # 创建DataFrame
    df = pd.DataFrame(data, columns=headers)
    df.to_csv(f"{filename}.csv", index=False)
    print(f"更新后的日程数据已保存到文件: {filename}.csv")

In [25]:
from langchain.chat_models import ChatOpenAI
# 读取csv中的日程表信息
sch_name = input("请输入日程表名称")
# sch_name = "schedule1"
schedule = pd.read_csv(f"{sch_name}.csv").to_string(index=False)
intent = input(f"您好，我是您的AI秘书，我将根据您的日程表{sch_name}为您服务。\n请问您有什么需要帮助的吗？")
# intent = " What tasks do I have now?"
while True:
    if intent != "结束对话":
        chat_prompt.format_prompt(schedule = schedule, intent = intent).to_messages()
        # 定义自然语言模型
        llm = OpenAI(model_name="gpt-3.5-turbo",max_tokens=1024,temperature=0.2)
        # 创建链
        from langchain.chains import LLMChain
        chain = LLMChain(llm=llm, prompt = chat_prompt)
        str = chain.run(schedule = schedule, intent = intent)
        answer,schedule = extract_answer_and_schedule(str)
        print(answer)
        intent = input("若需结束对话，请输入""结束对话""")
    else:
        print("对话已结束")
        parse_string_to_dataframe(schedule,sch_name)
        break

c:\Users\apple\anaconda3\lib\site-packages\langchain\llms\openai.py:169: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
c:\Users\apple\anaconda3\lib\site-packages\langchain\llms\openai.py:696: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


您今天有两个日程安排，分别是午餐和会议。请您尽快确定午餐地点，并提前到达会议室准备开会。如果您需要进行任何修改，请随时告诉我。
对话已结束
更新后的日程数据已保存到文件: schedule1.csv


In [29]:
# 输出最后保存的日程表
print(schedule)

事项         时间    地点                    备注
午餐 2023-05-12 12:30  待定 和朋友一起吃午餐，地点还没有定好.
会议 2023-05-12 14:00 办公室           持续两个小时.
打羽毛球 2023-05-12 20:00 学校体育馆     和室友一起打羽毛球...
